In [2]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
from crawlers.models import *
# from crawlers.finlab.pioneers import *
from crawlers.finlab.import_tools import add_to_sql,month_range
import time

In [3]:
class CrawlBrokerTradeTW:
    
    def __init__(self,start_date):
        self.start_date = start_date
        self.start_date_str = start_date.strftime("%Y-%m-%d")
        self.target_name = "台股分點進出資訊"
        self.format = "time_series"
        
    
    def check_trade_day(self):
        stock_range=CrawlStockPriceTW(self.start_date)
        try:
            df=pd.concat([stock_range.crawl_sii(),stock_range.crawl_otc()])['stock_id'].values
            return df
        except ValueError:
            return None
        
    def broker_trade(self,stock_id):
        print(stock_id)
        url='https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco.djhtm?a='+stock_id+'&e='+self.start_date_str+'&f='+self.start_date_str
        r = requests.post(url)
        html_df = pd.read_html(StringIO(r.text))
        df = pd.DataFrame(html_df[2])
        #holiday
        if len(df)<9:
            return  None
        df.columns=df.iloc[5]
        buy_net_avg_cost=float(df[df['買超券商']=='平均買超成本']['買超'].values[0])
        sell_net_avg_cost=float(df[df['賣超券商']=='平均賣超成本']['賣超'].values[0])
        df=df.iloc[6:-3]
        buy_side=df.iloc[:,:5]
        buy_side=buy_side.rename(columns={'買超券商':'broker_name','買進':'buy_num',
                                          '賣出':'sell_num','買超':'net_bs',
                                          '佔成交比重':'transactions_pt'})

        sell_side=df.iloc[:,5:]
        sell_side=sell_side.rename(columns={'賣超券商':'broker_name','買進':'buy_num',
                                            '賣出':'sell_num','賣超':'net_bs',
                                            '佔成交比重':'transactions_pt'})

        df_all=pd.concat([buy_side,sell_side],sort=False).dropna()
        df_all.iloc[:,1:]=df_all.iloc[:,1:].apply(lambda s: pd.to_numeric(s.str.replace('%',''), errors="coerce"))
        df_all['net_bs']=df_all['buy_num']-df_all['sell_num']
        df_all['net_bs_cost']=[i*buy_net_avg_cost if i>0 else i*sell_net_avg_cost for i in df_all['net_bs']]
        df_all['date'] = pd.to_datetime(self.start_date)
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s.replace('證券','')).apply(lambda s:s.replace('(牛牛牛)','犇'))
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s if '停' not in s else s[:s.index('-')])
        df_all['stock_id']=df_all['broker_name'].apply(lambda s:stock_id+'-'+s)
        AddToSQL.add_to_sql(BrokerTradeTW, df_all, 'broker_name')
        
        return  df_all

    def crawl_main(self):
        crawl_list=self.check_trade_day()
        if crawl_list is not None:
            for stock_id in crawl_list[:5]:
                 self.broker_trade(stock_id)
        else:
            pass



# a=CrawlBrokerTradeTW(datetime.datetime(2020,4,23))
# a.broker_trade('00774B')
# df=broker_trade(stock_id,datetime.datetime(2020,4,23),end_time=None)
# df

a=CrawlBrokerTradeTW(datetime.datetime(2018,1,10))
a.broker_trade('6263')
# a.crawl_main()

6263


/Users/benbilly3/Finlab_Django/venv/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


update  Stock_id:6263-富邦-陽明
update  Stock_id:6263-日盛
update  Stock_id:6263-亞東-板橋
update  Stock_id:6263-永全-南崁
update  Stock_id:6263-亞東-高雄
update  Stock_id:6263-兆豐-大安
update  Stock_id:6263-凱基-汐止
update  Stock_id:6263-日盛-板橋
update  Stock_id:6263-美林
update  Stock_id:6263-國泰-館前
update  Stock_id:6263-凱基
update  Stock_id:6263-新百王
update  Stock_id:6263-港商德意志
update  Stock_id:6263-亞東
update  Stock_id:6263-群益金鼎-新竹
update  Stock_id:6263-凱基-建成
update  Stock_id:6263-群益金鼎-天母
update  Stock_id:6263-元大-新莊
update  Stock_id:6263-凱基-竹科
update  Stock_id:6263-元大-中壢
update  Stock_id:6263-德信
update  Stock_id:6263-元大-松山
update  Stock_id:6263-台新
update  Stock_id:6263-犇亞-鑫豐
update  Stock_id:6263-日盛-永康
update  Stock_id:6263-大慶-楊梅
update  Stock_id:6263-第一金-桃園
update  Stock_id:6263-兆豐-城中
update  Stock_id:6263-凱基-竹北
update  Stock_id:6263-元大-虎尾
Finish <class 'crawlers.models.BrokerTradeTW'>date:2018-01-10T00:00:00.000000000 bulk_create:0
Finish <class 'crawlers.models.BrokerTradeTW'>date:2018-01-10T00:00:00.000000000

5,broker_name,buy_num,sell_num,net_bs,transactions_pt,net_bs_cost,date,stock_id
6,富邦-陽明,40,0,40,10.72,2574.40,2018-01-10,6263-富邦-陽明
7,日盛,17,0,17,4.56,1094.12,2018-01-10,6263-日盛
8,亞東-板橋,15,1,14,3.75,901.04,2018-01-10,6263-亞東-板橋
9,永全-南崁,11,0,11,2.95,707.96,2018-01-10,6263-永全-南崁
10,亞東-高雄,10,0,10,2.68,643.60,2018-01-10,6263-亞東-高雄
11,兆豐-大安,10,0,10,2.68,643.60,2018-01-10,6263-兆豐-大安
12,凱基-汐止,11,1,10,2.68,643.60,2018-01-10,6263-凱基-汐止
13,日盛-板橋,10,2,8,2.14,514.88,2018-01-10,6263-日盛-板橋
14,美林,7,0,7,1.88,450.52,2018-01-10,6263-美林
15,國泰-館前,7,0,7,1.88,450.52,2018-01-10,6263-國泰-館前


In [ ]:
df=StockPriceTW.objects.filter(date=datetime.datetime(2020,4,24))

In [ ]:
# [i['stock_id'] for i in df.values('stock_id')]
stock_range=CrawlStockPriceTW(date=datetime.datetime(2020,4,25))
try:
    df=pd.concat([stock_range.crawl_sii(),stock_range.crawl_otc()])['stock_id'].values
except ValueError:
    return None

In [ ]:
add_to_sql(BrokerTradeTW, df, fk_columns=['broker_name'],fk_corr_columns=['broker_name'])

In [ ]:
BrokerInfoTW.objects.filter(broker_name='臺銀')

# 匯入foreign,測試

In [ ]:
model_name=BrokerTradeTW

fk_field_names = [field.name for field in model_name._meta.fields
                             if isinstance(field, models.ForeignKey)]

fk_field_names

In [ ]:
fk_remote=[model_name._meta.get_field(i).remote_field.model  for i in fk_field_names]
fk_remote

In [ ]:
model_name=BrokerTradeTW
fk_columns='broker_name'

def fk_import(model_name,item,*fk_columns):
    fk_field_names = [field.name for field in model_name._meta.fields
                      if isinstance(field, models.ForeignKey)]
    fk_remote = [model_name._meta.get_field(i).remote_field.model for i in fk_field_names]
    fk_obj = [m.objects.get(**{n: item[n]}) for m, n in zip(fk_remote, fk_columns)]
    fk_create_data = dict((m, n) for m, n in zip(fk_field_names, fk_obj))
    return  fk_create_data


fk_import(model_name,df.iloc[0],fk_columns)

In [ ]:
model_name=BrokerTradeTW
fk_columns='broker_name'

def fk_update(model_name,item,*fk_columns):
    fk_field_names = [field.name for field in model_name._meta.fields
                      if isinstance(field, models.ForeignKey)]
    fk_remote = [model_name._meta.get_field(i).remote_field.model for i in fk_field_names]
    fk_obj = [m.objects.get(**{n: item[n]}) for m, n in zip(fk_remote, fk_columns)]

    return fk_obj 


fk_update(model_name,df.iloc[0],fk_columns)

In [ ]:
s=datetime.datetime(2020,5,10)
e= datetime.datetime(2020,5,15)
e.month
if e.month is not s.month:
    print('yes')
else:
    print('no')
# month_range(s,e)